In [ ]:
import copy
import numpy as np
import os
import re
import importlib
import sys
import pandas as pd
import utils as uti
def reload():
    importlib.reload(uti)

In [ ]:
src = os.getcwd()
text_path = os.path.join(src,'data','output')

**Preparar listados de palabras y frases a identificar después**

Estos listados los diseñamos manualmente en Excel.

In [ ]:
df_lists = pd.read_csv('data/input/listado.csv',delimiter=';')
df_lists.head(5)

<ipython-input-10-4a0d4a6ddbc0>:1: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_lists = pd.read_csv('data/input/listado.csv',delimiter=';')


,Listado marcadores discursivos,Listado Pronombres personales,Listado conjunciones y locuciones conjuntivas,Listado palabras que NO tienen contenido,Listado signos puntuacion,a,Unnamed: 6
0,NaN,yo,y,a,.,a causa de,NaN
1,NaN,tú,e,ante,",",a condición de,NaN
2,NaN,usted,ni,bajo,;,a condición de que,Palabras sin contenido:
3,así,él,que,cabe,:,a consecuencia de,"preposiciones, conjunciones, marcadores y dete..."
4,NaN,ella,o,con,¡,a continuación,NaN


In [ ]:
ls_marcadores = uti.prepare_list(df_lists,'Listado marcadores discursivos')
ls_conjunciones = uti.prepare_list(df_lists,'Listado conjunciones y locuciones conjuntivas')
ls_pronombres = uti.prepare_list(df_lists,'Listado Pronombres personales')
ls_puntuacion = uti.prepare_list(df_lists,'Listado signos puntuacion')
ls_content_words = uti.prepare_list(df_lists,'Listado palabras que NO tienen contenido')

**Preparar diccionario de los paths a los textos**

In [ ]:
texts = {i[:-4]:os.path.join('data/input',i) for i in os.listdir('data/input') if 'TO' not in i and i.endswith('.txt')}
texts

{'TH4': 'data/input/TH4.txt',
 'PE4': 'data/input/PE4.txt',
 'TH2': 'data/input/TH2.txt',
 'PE1': 'data/input/PE1.txt',
 'PE3': 'data/input/PE3.txt',
 'PE2': 'data/input/PE2.txt',
 'TH5': 'data/input/TH5.txt',
 'TH1': 'data/input/TH1.txt',
 'PE5': 'data/input/PE5.txt',
 'TH6': 'data/input/TH6.txt',
 'TH3': 'data/input/TH3.txt',
 'PE6': 'data/input/PE6.txt'}

In [ ]:
translation = {'unique_words':             'Numero total de palabras unicas',
 'total_words':                            'Numero total de palabras',
 'unique_ratio':                           'Porcentaje de palabras unicas',
 'content_word_ratio_per_phrase_average':  'Porcentage de palabras con contenido',
 'total_average_characters_per_word':      'Media de caracteres por palabra',
 'mean_number_phrases':                    'Media de palabras por frase',
 'number_phrases':                         'Numero de frases',
 'len_text':                               'Longitud total del texto (caracteres)',
'mean_pronombres':                         'Media pronombres',
'pctg_pronombres':                         'Porcentaje pronombres',
'mean_content_word':                         'Media palabras con contenido',
'mean_conjunciones':                         'Media conjunciones',
'pctg_conjunciones':                         'Porcentaje conjunciones',
'mean_marcadores_discursivos':                         'Media marcadores discursivos',
'pctg_marcadores_discursivos':                         'Porcentaje marcadores discursivos',
'mean_punctuations':                         'Media signos de puntuacion',
'pctg_punctuations':                         'Porcentaje signos de puntuacion',
}

**Declaramos la función principal**

Se ocupa de llamar a las diferentes funciones de cada apartado y da como resultado un diccionario con los resultados.

Puede autogenerar unos excels con los resultados de cada texto

In [ ]:
def evaluate_text(name: str, path, save: bool=False) -> dict:
    raw_t, clean_t = uti.read(path)
    punctuations = uti.find_punctuation_in_list(raw_t,ls_puntuacion)

    l = uti.simplificacion_1_1(clean_t,ls_content_words)
    (unique_words, total_words, unique_ratio, number_non_content_words, number_word_in_phrase,
     non_content_word_ratio_per_phrase, non_content_word_ratio_per_phrase_average, average_characters_per_word,
     total_average_characters_per_word) = l

    number_word_in_phrase, mean_number_phrases, number_phrases = uti.simplificacion_1_2(clean_t)

    number_marcadores_discursivos = uti.explicitacion_2_1(clean_t, ls_marcadores)

    number_conjunciones = uti.explicitacion_2_2(clean_t, ls_conjunciones)

    number_characters_per_phrase, len_text = uti.explicitacion_2_3(clean_t)

    number_pronombres = uti.explicitacion_2_4(clean_t, ls_pronombres)

    dic ={'punctuations':punctuations,
        'unique_words' : unique_words,
    'total_words' : total_words,
    'unique_ratio' : unique_ratio,
    'number_non_content_words' : number_non_content_words,
    'number_word_in_phrase' : number_word_in_phrase,
    'non_content_word_ratio_per_phrase' : non_content_word_ratio_per_phrase,
    'non_content_word_ratio_per_phrase_average':non_content_word_ratio_per_phrase_average,
    'average_characters_per_word':average_characters_per_word,
    'total_average_characters_per_word':total_average_characters_per_word,
    'number_word_in_phrase' : number_word_in_phrase,
    'mean_number_phrases' : mean_number_phrases,
    'number_phrases' : number_phrases,
    'number_marcadores_discursivos' : number_marcadores_discursivos,
    'number_conjunciones' : number_conjunciones,
    'number_characters_per_phrase' : number_characters_per_phrase,
    'len_text' : len_text,
    'number_pronombres' : number_pronombres,}
    dic = uti.final_statistics(dic)
    dic_stats = {key:value for key,value in dic.items() if not isinstance(value,(list,np.ndarray))}

    output_path = 'data/output'
    stats_path = os.path.join(output_path, f'{name}_stats.xlsx')
    phrase_path = os.path.join(output_path, f'{name}.xlsx')

    if save:
        pd.DataFrame.from_dict(dic_stats,orient='index').rename(index=translation,columns={0:name}).to_excel(stats_path)

    df = pd.DataFrame(clean_t,columns=['text'])
    for key,value in dic.items():
        if isinstance(value,(list, np.ndarray)):
            df[key]=value
    if save:
        df.to_excel(phrase_path)
    return dic

**Llamamos a la función principal en bucle para cada texto**

In [ ]:
for name, path in texts.items():
    dic_iter = evaluate_text(name, path, save=True)